In [1]:
import importlib
import dataset
importlib.reload(dataset)
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists
from libreco.data import split_by_ratio_chrono, DatasetPure
from libreco.algorithms import ALS
from sklearn.metrics import mean_squared_error

2022-07-19 11:53:47.203866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-19 11:53:47.203901: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
def save_predictions_from_pandas(res_path, predictions, index_pd):
    index_pd = index_pd.astype({'Prediction': 'float'})
    index_pd.iloc[:, 1] = predictions
    index_pd.to_csv(res_path, index=False, float_format='%.3f')

In [4]:
def save_predictions(res_path, predictions):
    test_pd = pd.read_csv('../data/sampleSubmission.csv')
    test_pd = test_pd.astype({'Prediction': 'float'})
    test_pd.iloc[:, 1] = predictions
    test_pd.to_csv(res_path, index=False, float_format='%.3f')

In [5]:
RANDOM_STATE = 58

scores = []

test_pd = pd.read_csv('../data/sampleSubmission.csv')
users_test, movies_test, _ = extract_users_movies_ratings_lists(test_pd)


for i in range(5):
    train_pd = pd.read_csv(f'../data_val_train_kfold/partition_{i}_train.csv')
    val_pd = pd.read_csv(f'../data_val_train_kfold/partition_{i}_val.csv')
    users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
    users_val, movies_val, ratings_val = extract_users_movies_ratings_lists(val_pd)
    train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
    val = pd.DataFrame({'user': users_val, 'item': movies_val, 'label': ratings_val})
    train_data, data_info = DatasetPure.build_trainset(train)
    eval_data = DatasetPure.build_evalset(val)
    
    score = eval_ensemble(i, train_data, data_info, users_val, movies_val, ratings_val, users_test, movies_test)
    print(score)
    scores.append(score)

NameError: name 'eval_ensemble' is not defined

In [6]:
def eval_ensemble(split, train_data, data_info, users_val, movies_val, ratings_val, users_test, movies_test):
    emb_sizes = [2,3,4,5,6,7,8,9,10,11,16,32,58]
    regs = [0.1, 0.5, 1, 5, 10]
    alphas = [0.01, 0.1, 1, 5, 8, 10, 100]
    david_val = np.zeros((len(emb_sizes) * len(regs) * len(alphas), len(users_val)))
    david_test = np.zeros((len(emb_sizes) * len(regs) * len(alphas), len(users_test)))

    for i, emb_size in enumerate(emb_sizes):
        for j, reg in enumerate(regs):
            for k, alpha in enumerate(alphas):
                giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                            reg=reg, alpha=alpha, seed=42)
                giorgio.fit(train_data, verbose=0, use_cg=False, n_threads=8, metrics=["rmse", "mae", "r2"])
                david_val[len(regs)*len(alphas)*i+len(alphas)*j+k] = np.array(giorgio.predict(user=users_val, item=movies_val))
                david_test[len(regs)*len(alphas)*i+len(alphas)*j+k] = np.array(giorgio.predict(user=users_test, item=movies_test))

    yhat_val = david_val.mean(axis=0)
    yhat_test = david_test.mean(axis=0)
    save_predictions_from_pandas(f'../results_ensemble/ALS/ALS_split_{split}_val_results.csv', yhat_val, val_pd)
    save_predictions(f'../results_ensemble/ALS/ALS_split_{split}_test_results.csv', yhat_test)
    
    return mean_squared_error(yhat_val, ratings_val, squared=False)

In [ ]:
# --------- generate submission -------
emb_sizes = [2,3,4,5,6,7,8,9,10,11,16,32,58]
regs = [0.1, 0.5, 1, 5, 10]
alphas = [0.01, 0.1, 1, 5, 8, 10, 100]
david = np.zeros((len(emb_sizes) * len(regs) * len(alphas), len(users_test)))
    
for i, emb_size in enumerate(emb_sizes):
    for j, reg in enumerate(regs):
        for k, alpha in enumerate(alphas):
            giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                        reg=reg, alpha=alpha, seed=42)
            giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8, metrics=["rmse", "mae", "r2"])
            david[len(regs)*len(alphas)*i+len(alphas)*j+k] = np.array(giorgio.predict(user=users_test, item=movies_test))
            
yhat = david.mean(axis=0)
save_predictions(f'../results_ensemble/ALS/ALS-predictedSubmission.csv', yhat)

Training start time: 2022-07-18 17:14:29
Epoch 1 elapsed: 0.042s
Epoch 2 elapsed: 0.050s
Epoch 3 elapsed: 0.024s
Epoch 4 elapsed: 0.024s
Epoch 5 elapsed: 0.024s
Training start time: 2022-07-18 17:14:30
Epoch 1 elapsed: 0.024s
Epoch 2 elapsed: 0.024s
Epoch 3 elapsed: 0.038s
Epoch 4 elapsed: 0.025s
Epoch 5 elapsed: 0.025s
Training start time: 2022-07-18 17:14:31
Epoch 1 elapsed: 0.027s
Epoch 2 elapsed: 0.026s
Epoch 3 elapsed: 0.053s
Epoch 4 elapsed: 0.031s
Epoch 5 elapsed: 0.024s
Training start time: 2022-07-18 17:14:32
Epoch 1 elapsed: 0.030s
Epoch 2 elapsed: 0.024s
Epoch 3 elapsed: 0.025s
Epoch 4 elapsed: 0.024s
Epoch 5 elapsed: 0.033s
Training start time: 2022-07-18 17:14:33
Epoch 1 elapsed: 0.024s
Epoch 2 elapsed: 0.024s
Epoch 3 elapsed: 0.024s
Epoch 4 elapsed: 0.059s
Epoch 5 elapsed: 0.025s
Training start time: 2022-07-18 17:14:34
Epoch 1 elapsed: 0.076s
Epoch 2 elapsed: 0.064s
Epoch 3 elapsed: 0.045s
Epoch 4 elapsed: 0.080s
Epoch 5 elapsed: 0.027s
Training start time: 2022-07-18 17

Training start time: 2022-07-18 17:14:57
Epoch 1 elapsed: 0.106s
Epoch 2 elapsed: 0.139s
Epoch 3 elapsed: 0.097s
Epoch 4 elapsed: 0.128s
Epoch 5 elapsed: 0.115s
Training start time: 2022-07-18 17:14:59
Epoch 1 elapsed: 0.081s
Epoch 2 elapsed: 0.090s
Epoch 3 elapsed: 0.095s
Epoch 4 elapsed: 0.099s
Epoch 5 elapsed: 0.126s
Training start time: 2022-07-18 17:15:00
Epoch 1 elapsed: 0.076s
Epoch 2 elapsed: 0.054s
Epoch 3 elapsed: 0.114s
Epoch 4 elapsed: 0.110s
Epoch 5 elapsed: 0.072s
Training start time: 2022-07-18 17:15:01
Epoch 1 elapsed: 0.047s
Epoch 2 elapsed: 0.032s
Epoch 3 elapsed: 0.025s
Epoch 4 elapsed: 0.026s
Epoch 5 elapsed: 0.034s
Training start time: 2022-07-18 17:15:02
Epoch 1 elapsed: 0.029s
Epoch 2 elapsed: 0.031s
Epoch 3 elapsed: 0.026s
Epoch 4 elapsed: 0.026s
Epoch 5 elapsed: 0.042s
Training start time: 2022-07-18 17:15:05
Epoch 1 elapsed: 0.108s
Epoch 2 elapsed: 0.112s
Epoch 3 elapsed: 0.109s
Epoch 4 elapsed: 0.097s
Epoch 5 elapsed: 0.101s
Training start time: 2022-07-18 17

Epoch 3 elapsed: 0.136s
Epoch 4 elapsed: 0.127s
Epoch 5 elapsed: 0.134s
Training start time: 2022-07-18 17:15:35
Epoch 1 elapsed: 0.130s
Epoch 2 elapsed: 0.109s
Epoch 3 elapsed: 0.126s
Epoch 4 elapsed: 0.099s
Epoch 5 elapsed: 0.104s
Training start time: 2022-07-18 17:15:36
Epoch 1 elapsed: 0.115s
Epoch 2 elapsed: 0.100s
Epoch 3 elapsed: 0.119s
Epoch 4 elapsed: 0.116s
Epoch 5 elapsed: 0.100s
Training start time: 2022-07-18 17:15:38
Epoch 1 elapsed: 0.114s
Epoch 2 elapsed: 0.120s
Epoch 3 elapsed: 0.122s
Epoch 4 elapsed: 0.120s
Epoch 5 elapsed: 0.121s
Training start time: 2022-07-18 17:15:39
Epoch 1 elapsed: 0.108s
Epoch 2 elapsed: 0.114s
Epoch 3 elapsed: 0.121s
Epoch 4 elapsed: 0.106s
Epoch 5 elapsed: 0.124s
Training start time: 2022-07-18 17:15:41
Epoch 1 elapsed: 0.072s
Epoch 2 elapsed: 0.040s
Epoch 3 elapsed: 0.046s
Epoch 4 elapsed: 0.040s
Epoch 5 elapsed: 0.036s
Training start time: 2022-07-18 17:15:42
Epoch 1 elapsed: 0.093s
Epoch 2 elapsed: 0.114s
Epoch 3 elapsed: 0.135s
Epoch 4 el

Epoch 5 elapsed: 0.037s
Training start time: 2022-07-18 17:16:09
Epoch 1 elapsed: 0.036s
Epoch 2 elapsed: 0.090s
Epoch 3 elapsed: 0.056s
Epoch 4 elapsed: 0.036s
Epoch 5 elapsed: 0.035s
Training start time: 2022-07-18 17:16:10
Epoch 1 elapsed: 0.040s
Epoch 2 elapsed: 0.037s
Epoch 3 elapsed: 0.053s
Epoch 4 elapsed: 0.037s
Epoch 5 elapsed: 0.045s
Training start time: 2022-07-18 17:16:11
Epoch 1 elapsed: 0.037s
Epoch 2 elapsed: 0.037s
Epoch 3 elapsed: 0.042s
Epoch 4 elapsed: 0.036s
Epoch 5 elapsed: 0.037s
Training start time: 2022-07-18 17:16:12
Epoch 1 elapsed: 0.051s
Epoch 2 elapsed: 0.039s
Epoch 3 elapsed: 0.036s
Epoch 4 elapsed: 0.036s
Epoch 5 elapsed: 0.037s
Training start time: 2022-07-18 17:16:13
Epoch 1 elapsed: 0.050s
Epoch 2 elapsed: 0.037s
Epoch 3 elapsed: 0.048s
Epoch 4 elapsed: 0.039s
Epoch 5 elapsed: 0.044s
Training start time: 2022-07-18 17:16:14
Epoch 1 elapsed: 0.041s
Epoch 2 elapsed: 0.037s
Epoch 3 elapsed: 0.049s
Epoch 4 elapsed: 0.036s
Epoch 5 elapsed: 0.037s
Training s

Epoch 3 elapsed: 0.088s
Epoch 4 elapsed: 0.056s
Epoch 5 elapsed: 0.054s
Training start time: 2022-07-18 17:16:36
Epoch 1 elapsed: 0.036s
Epoch 2 elapsed: 0.036s
Epoch 3 elapsed: 0.037s
Epoch 4 elapsed: 0.038s
Epoch 5 elapsed: 0.040s
Training start time: 2022-07-18 17:16:37
Epoch 1 elapsed: 0.036s
Epoch 2 elapsed: 0.037s
Epoch 3 elapsed: 0.036s
Epoch 4 elapsed: 0.038s
Epoch 5 elapsed: 0.036s
Training start time: 2022-07-18 17:16:38
Epoch 1 elapsed: 0.044s
Epoch 2 elapsed: 0.045s
Epoch 3 elapsed: 0.045s
Epoch 4 elapsed: 0.046s
Epoch 5 elapsed: 0.044s
Training start time: 2022-07-18 17:16:39
Epoch 1 elapsed: 0.091s
Epoch 2 elapsed: 0.058s
Epoch 3 elapsed: 0.047s
Epoch 4 elapsed: 0.065s
Epoch 5 elapsed: 0.044s
Training start time: 2022-07-18 17:16:40
Epoch 1 elapsed: 0.047s
Epoch 2 elapsed: 0.078s
Epoch 3 elapsed: 0.069s
Epoch 4 elapsed: 0.048s
Epoch 5 elapsed: 0.060s
Training start time: 2022-07-18 17:16:41
Epoch 1 elapsed: 0.058s
Epoch 2 elapsed: 0.044s
Epoch 3 elapsed: 0.044s
Epoch 4 el

...

Epoch 2 elapsed: 0.126s
Epoch 3 elapsed: 0.129s
Epoch 4 elapsed: 0.125s
Epoch 5 elapsed: 0.148s
Training start time: 2022-07-18 17:21:19
Epoch 1 elapsed: 0.138s
Epoch 2 elapsed: 0.117s
Epoch 3 elapsed: 0.118s
Epoch 4 elapsed: 0.121s
Epoch 5 elapsed: 0.119s
Training start time: 2022-07-18 17:21:21
Epoch 1 elapsed: 0.111s
Epoch 2 elapsed: 0.130s
Epoch 3 elapsed: 0.118s
Epoch 4 elapsed: 0.122s
Epoch 5 elapsed: 0.139s
Training start time: 2022-07-18 17:21:22
Epoch 1 elapsed: 0.141s
Epoch 2 elapsed: 0.125s
Epoch 3 elapsed: 0.135s
Epoch 4 elapsed: 0.128s
Epoch 5 elapsed: 0.117s
Training start time: 2022-07-18 17:21:24
Epoch 1 elapsed: 0.110s
Epoch 2 elapsed: 0.112s
Epoch 3 elapsed: 0.115s
Epoch 4 elapsed: 0.124s
Epoch 5 elapsed: 0.130s
Training start time: 2022-07-18 17:21:25
Epoch 1 elapsed: 0.130s
Epoch 2 elapsed: 0.142s
Epoch 3 elapsed: 0.113s
Epoch 4 elapsed: 0.115s
Epoch 5 elapsed: 0.126s
Training start time: 2022-07-18 17:21:27
Epoch 1 elapsed: 0.120s
Epoch 2 elapsed: 0.126s
Epoch 3 el

Epoch 4 elapsed: 0.210s
Epoch 5 elapsed: 0.214s
Training start time: 2022-07-18 17:21:59
Epoch 1 elapsed: 0.212s
Epoch 2 elapsed: 0.235s
Epoch 3 elapsed: 0.250s
Epoch 4 elapsed: 0.235s
Epoch 5 elapsed: 0.214s
Training start time: 2022-07-18 17:22:01
Epoch 1 elapsed: 0.225s
Epoch 2 elapsed: 0.217s
Epoch 3 elapsed: 0.230s
Epoch 4 elapsed: 0.233s
Epoch 5 elapsed: 0.235s
Training start time: 2022-07-18 17:22:03
Epoch 1 elapsed: 0.231s
Epoch 2 elapsed: 0.220s
Epoch 3 elapsed: 0.232s
Epoch 4 elapsed: 0.210s
Epoch 5 elapsed: 0.220s
Training start time: 2022-07-18 17:22:05
Epoch 1 elapsed: 0.251s
Epoch 2 elapsed: 0.210s
Epoch 3 elapsed: 0.209s
Epoch 4 elapsed: 0.208s
Epoch 5 elapsed: 0.228s
Training start time: 2022-07-18 17:22:07
Epoch 1 elapsed: 0.231s
Epoch 2 elapsed: 0.217s
Epoch 3 elapsed: 0.248s
Epoch 4 elapsed: 0.233s
Epoch 5 elapsed: 0.208s
Training start time: 2022-07-18 17:22:09
Epoch 1 elapsed: 0.199s
Epoch 2 elapsed: 0.209s
Epoch 3 elapsed: 0.217s
Epoch 4 elapsed: 0.209s
Epoch 5 el

Training start time: 2022-07-18 17:22:47
Epoch 1 elapsed: 0.233s
Epoch 2 elapsed: 0.208s
Epoch 3 elapsed: 0.222s
Epoch 4 elapsed: 0.226s
Epoch 5 elapsed: 0.207s
Training start time: 2022-07-18 17:22:49
Epoch 1 elapsed: 0.222s
Epoch 2 elapsed: 0.233s
Epoch 3 elapsed: 0.230s
Epoch 4 elapsed: 0.250s
Epoch 5 elapsed: 0.205s
Training start time: 2022-07-18 17:22:51
Epoch 1 elapsed: 0.197s
Epoch 2 elapsed: 0.214s
Epoch 3 elapsed: 0.209s
Epoch 4 elapsed: 0.217s
Epoch 5 elapsed: 0.213s
Training start time: 2022-07-18 17:22:53
Epoch 1 elapsed: 0.229s
Epoch 2 elapsed: 0.275s
Epoch 3 elapsed: 0.237s
Epoch 4 elapsed: 0.244s
Epoch 5 elapsed: 0.238s
Training start time: 2022-07-18 17:22:55
Epoch 1 elapsed: 0.231s
Epoch 2 elapsed: 0.249s
Epoch 3 elapsed: 0.269s
Epoch 4 elapsed: 0.236s
Epoch 5 elapsed: 0.239s
Training start time: 2022-07-18 17:22:57
Epoch 1 elapsed: 0.256s
Epoch 2 elapsed: 0.239s
Epoch 3 elapsed: 0.252s
Epoch 4 elapsed: 0.232s
Epoch 5 elapsed: 0.243s
Training start time: 2022-07-18 17

Epoch 2 elapsed: 0.232s
Epoch 3 elapsed: 0.241s
Epoch 4 elapsed: 0.270s
Epoch 5 elapsed: 0.256s
Training start time: 2022-07-18 17:23:40
Epoch 1 elapsed: 0.244s
Epoch 2 elapsed: 0.258s
Epoch 3 elapsed: 0.232s
Epoch 4 elapsed: 0.244s
Epoch 5 elapsed: 0.247s
Training start time: 2022-07-18 17:23:43
Epoch 1 elapsed: 0.223s
Epoch 2 elapsed: 0.231s
Epoch 3 elapsed: 0.246s
Epoch 4 elapsed: 0.244s
Epoch 5 elapsed: 0.236s
Training start time: 2022-07-18 17:23:45
Epoch 1 elapsed: 0.224s
Epoch 2 elapsed: 0.234s
Epoch 3 elapsed: 0.233s
Epoch 4 elapsed: 0.233s
Epoch 5 elapsed: 0.239s
Training start time: 2022-07-18 17:23:47
Epoch 1 elapsed: 0.303s
Epoch 2 elapsed: 0.252s
Epoch 3 elapsed: 0.232s
Epoch 4 elapsed: 0.255s
Epoch 5 elapsed: 0.235s
Training start time: 2022-07-18 17:23:49
Epoch 1 elapsed: 0.226s
Epoch 2 elapsed: 0.237s
Epoch 3 elapsed: 0.233s
Epoch 4 elapsed: 0.237s
Epoch 5 elapsed: 0.245s
Training start time: 2022-07-18 17:23:51
Epoch 1 elapsed: 0.236s
Epoch 2 elapsed: 0.231s
Epoch 3 el

Epoch 4 elapsed: 1.215s
Epoch 5 elapsed: 1.217s
Training start time: 2022-07-18 17:25:40
Epoch 1 elapsed: 1.178s
Epoch 2 elapsed: 1.226s
Epoch 3 elapsed: 1.212s
Epoch 4 elapsed: 1.253s
Epoch 5 elapsed: 1.208s
Training start time: 2022-07-18 17:25:47
Epoch 1 elapsed: 1.212s
Epoch 2 elapsed: 1.215s
Epoch 3 elapsed: 1.258s
Epoch 4 elapsed: 1.202s
Epoch 5 elapsed: 1.190s
Training start time: 2022-07-18 17:25:54
Epoch 1 elapsed: 1.216s
Epoch 2 elapsed: 1.278s
Epoch 3 elapsed: 1.212s
Epoch 4 elapsed: 1.207s
Epoch 5 elapsed: 1.251s
Training start time: 2022-07-18 17:26:01
Epoch 1 elapsed: 1.202s
Epoch 2 elapsed: 1.211s
Epoch 3 elapsed: 1.216s
Epoch 4 elapsed: 1.221s
Epoch 5 elapsed: 1.230s
Training start time: 2022-07-18 17:26:08
Epoch 1 elapsed: 1.195s
Epoch 2 elapsed: 1.206s
Epoch 3 elapsed: 1.189s
Epoch 4 elapsed: 1.198s
Epoch 5 elapsed: 1.237s
Training start time: 2022-07-18 17:26:15
Epoch 1 elapsed: 1.244s
Epoch 2 elapsed: 1.258s
Epoch 3 elapsed: 1.309s
Epoch 4 elapsed: 1.334s
Epoch 5 el